In [1]:
import pandas as pd
import numpy as np

from scipy.optimize import curve_fit

import plotly.express as px

## Obtain and save data from the web

In [2]:
# # the service URL
# livechart = "https://nds.iaea.org/relnsd/v1/data?"

# # There have been cases in which the service returns an HTTP Error 403: Forbidden
# # use this workaround
# import urllib.request
# def lc_pd_dataframe(url):
#     req = urllib.request.Request(url)
#     req.add_header('User-Agent''','' 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
#     return pd.read_csv(urllib.request.urlopen(req))

In [3]:
# ground_states_df = lc_pd_dataframe(livechart + "fields=ground_states&nuclides=all")
# ground_states_df = ground_states_df[['z', 'n', 'symbol', 'half_life']]
# ground_states_df = ground_states_df[ground_states_df['half_life'] == 'STABLE']
# ground_states_df = ground_states_df.copy()
# ground_states_df['a'] = ground_states_df['z'] + ground_states_df['n']
# ground_states_df['symbol'] = ground_states_df['symbol'].str.lower()
# ground_states_df['name'] = ground_states_df['a'].astype(str).str.cat(ground_states_df['symbol'])
# names = ground_states_df['name'].values

In [4]:
# levels_dfs = [lc_pd_dataframe(livechart + f"fields=levels&nuclides={name}") for name in names]
# levels_dfs =  pd.concat(levels_dfs, ignore_index=True)

# print(levels_dfs.columns)
# print(levels_dfs.head())

In [5]:
# levels_dfs.to_csv("levels.csv", index=False)

## Load saved data

In [6]:
dtypes = {
    'z': 'UInt8',
    'n': 'UInt8',
    'symbol': 'string',
    'idx': 'UInt16',
    'energy_shift': 'category',
    'energy': 'Float64',
    'unc_e': 'Float64',
    'ripl_shift': 'Float64',
    'jp': 'string',
    'jp_order': 'UInt8',
    'half_life': 'string',
    'operator_hl': 'string',
    'unc_hl': 'string',
    'unit_hl': 'category',
    'half_life_sec': 'Float64',
    'unc_hl.1': 'Float64',
    'decay_1': 'string',
    'decay_1_%': 'Float64',
    'unc_1': 'Float64',
    'decay_2': 'string',
    'decay_2_%': 'Float64',
    'unc_2': 'Float64',
    'decay_3': 'string',
    'decay_3_%': 'Float64',
    'unc_3': 'Float64',
    'isospin': 'string',
    'magnetic_dipole': 'Float64',
    'unc_mn': 'Float64',
    'electric_quadrupole': 'Float64',
    'unc_eq': 'Float64',
    'ENSDF_publication_cut-off': 'string',
    'ENSDF_authors': 'string',
    'Extraction_date': 'string'
}
parse_dates = ['ENSDF_publication_cut-off', 'Extraction_date']


df = pd.read_csv('levels.csv', parse_dates=parse_dates, dtype=dtypes)


df.loc[df['half_life'] == 'STABLE', 'half_life'] = 'inf'
df['half_life'] = df['half_life'].astype('Float64')

df['a'] = df['z'] + df['n']

df = df[['symbol', 'a', 'z', 'n', 'idx', 'energy', 'energy_shift', 'ripl_shift', 'jp', 'jp_order']]
df['energy'] = df['energy'] / 1000 #MeV
df['ripl_shift'] = df['ripl_shift'] / 1000 #MeV
#TODO also take into account energy uncertainty

df['beta'] = (df['n'].astype('Float64') - df['z'].astype('Float64')) / df['a'].astype('Float64')

df = df.set_index(['z', 'a'], drop=True)

df.head()

symbol  n  idx  energy energy_shift  ripl_shift    jp  jp_order      beta
z a                                                                          
1 1      H  0    0     0.0          NaN        <NA>  1/2+         1      -1.0
  2      H  1    0     0.0          NaN        <NA>    1+         1       0.0
2 3     He  1    0     0.0          NaN        <NA>  1/2+         1 -0.333333
  4     He  2    0     0.0          NaN        <NA>    0+         1       0.0
  4     He  2    1   20.21          NaN        <NA>    0+         2       0.0

In [7]:
# Filter out shifts
#TODO take into account energy shifts with ripl_shift
#TODO take into account energy shifts without ripl_shift

df = df[df['energy_shift'].isna()]
df = df.drop(['energy_shift', 'ripl_shift'], axis='columns')
df.head()

symbol  n  idx  energy    jp  jp_order      beta
z a                                                 
1 1      H  0    0     0.0  1/2+         1      -1.0
  2      H  1    0     0.0    1+         1       0.0
2 3     He  1    0     0.0  1/2+         1 -0.333333
  4     He  2    0     0.0    0+         1       0.0
  4     He  2    1   20.21    0+         2       0.0

In [8]:
# Filter out not enough levels
# Just removes 1H, 2H, and 3He

df = df.loc[df.groupby(level=df.index.names).size() >= 3]

# Filter out uncertain jp

df = df[df['jp'].str.fullmatch(r'^[0-9]+(/[0-9]+)?[+-]$')]

# Extract j and p

df['j'] = df['jp'].str[:-1]
df['p'] = df['jp'].str[-1]

odd_spins = df['j'].str.contains('/')
even_spins = ~odd_spins

df.loc[odd_spins,'j_float'] = df[odd_spins]['j'].str.split('/').apply(lambda x: float(x[0]) / float(x[1]))
df.loc[even_spins,'j_float'] = df[even_spins]['j'].astype(float)

df['j_evenness'] = pd.Series(data=pd.NA, dtype='boolean')
df.loc[even_spins,'j_evenness'] = (df[even_spins]['j'].str.split('/', expand=True)[0].astype(int) % 2 == 0)
#TODO similar for odd_spins

df['p_bit'] = df['p'].str.fullmatch(r'\-').astype(int)

# Search

### Quadrupole, positive, first band 

In [17]:
def get_rot_energy(l_over_two, energy_quantum):
    l = l_over_two*2
    return l*(l+1) * energy_quantum

def fit(group, func):

    x = group['quanta']
    y = group['energy']

    # fit
    popt, pcov = curve_fit(func, x, y)

    # prediction
    y_pred = func(x, *popt)

    # r-squared
    ss_res = np.sum((y - y_pred)**2)
    ss_tot = np.sum((y - y.mean())**2)
    r2 = 1 - (ss_res / ss_tot)

    # results
    results = pd.DataFrame({'energy_quantum': popt[0], 'r2': r2, 'quanta': x, 'energy_pred': y_pred})

    # results = pd.Series(list(popt)+[r2], index=['energy_quantum', 'zero_point_energy', 'r2'])
    return  results

In [18]:
quad_pos_first = df[(df['jp_order'] == 1) & df['j_evenness'].fillna(False) & (df['p_bit'] == 0)]
quad_pos_first = quad_pos_first.loc[quad_pos_first.groupby(level=df.index.names).size() >= 3]
quad_pos_first['quanta'] = (quad_pos_first['j_float'].astype(int) // 2)

quad_pos_first_groups = quad_pos_first.groupby(level=df.index.names, as_index=False)

oscillator_fit = quad_pos_first_groups.apply(lambda group: fit(group, get_rot_energy)).droplevel(0)

In [19]:
min_r2 = 0.9

print("Oscillator osc:")
best_oscillator_fit = oscillator_fit[oscillator_fit['r2'] > min_r2]

print(best_oscillator_fit.groupby(by=best_oscillator_fit.index)['r2'].head(1))

Oscillator osc:
z   a  
3   6      0.922325
6   12     0.997976
8   18     0.956019
10  20     0.948610
    22     0.988582
             ...   
78  196    0.967662
    198    0.971478
80  196    0.981404
    202    0.979921
    204    0.992779
Name: r2, Length: 94, dtype: float64


In [20]:
#TODO plot r^2 on grid
#TODO change starting point

In [21]:
merged = quad_pos_first.merge(oscillator_fit, how='left', on=['z','a', 'quanta'])
best_osc_merged = merged[merged['r2'] > min_r2]

def get_best_r2(df, min_r2):
    return df[df['r2'] > min_r2]

# merged = merged.merge(rotator_fit, how='left', on=['z','a', 'idx'], suffixes=('_osc', '_rot'))
# lowest_merged = merged.groupby(['z','a']).head(3)
lowest_merged = merged
# best_rot_merged = lowest_merged[lowest_merged['r2_rot'] > min_r2]
# best_rot_merged

In [22]:
best_osc_merged

symbol    n  idx    energy  jp  jp_order      beta  j  p  j_float  \
z  a                                                                       
3  6       Li    3    2   3.56288  0+         1       0.0  0  +      0.0   
   6       Li    3    3     4.312  2+         1       0.0  2  +      2.0   
   6       Li    3    9      23.0  4+         1       0.0  4  +      4.0   
6  12       C    6    0       0.0  0+         1       0.0  0  +      0.0   
   12       C    6    1   4.43982  2+         1       0.0  2  +      2.0   
...       ...  ...  ...       ...  ..       ...       ... .. ..      ...   
80 202     Hg  122   34   1.98882  6+         1  0.207921  6  +      6.0   
   204     Hg  124    0       0.0  0+         1  0.215686  0  +      0.0   
   204     Hg  124    1  0.436552  2+         1  0.215686  2  +      2.0   
   204     Hg  124    2   1.12823  4+         1  0.215686  4  +      4.0   
   204     Hg  124   15   2.19101  6+         1  0.215686  6  +      6.0   

        j_evenness  p_bit  quanta  energy_quantum        r2  energy_pred  
z  a                                                                      
3  6          True      0       0        1.114385  0.922325     0.000000  
   6          True      0       1        1.114385  0.922325     6.686312  
   6          True      0       2        1.114385  0.922325    22.287706  
6  12         True      0       0        0.671190  0.997976     0.000000  
   12         True      0       1        0.671190  0.997976     4.027141  
...            ...    ...     ...             ...       ...          ...  
80 202        True      0       3        0.049348  0.979921     2.072624  
   204        True      0       0        0.053276  0.992779     0.000000  
   204        True      0       1        0.053276  0.992779     0.319654  
   204        True      0       2        0.053276  0.992779     1.065512  
   204        True      0       3        0.053276  0.992779     2.237575  

[746 rows x 16 columns]

In [23]:
merged

symbol    n  idx    energy   jp  jp_order      beta   j  p  j_float  \
z  a                                                                         
3  6       Li    3    2   3.56288   0+         1       0.0   0  +      0.0   
   6       Li    3    3     4.312   2+         1       0.0   2  +      2.0   
   6       Li    3    9      23.0   4+         1       0.0   4  +      4.0   
5  10       B    5    2   1.74005   0+         1       0.0   0  +      0.0   
   10       B    5    4   3.58713   2+         1       0.0   2  +      2.0   
...       ...  ...  ...       ...  ...       ...       ...  .. ..      ...   
82 208     Pb  126   20  4.323946   4+         1  0.211538   4  +      4.0   
   208     Pb  126   23  4.423647   6+         1  0.211538   6  +      6.0   
   208     Pb  126   26  4.610748   8+         1  0.211538   8  +      8.0   
   208     Pb  126   38   4.89523  10+         1  0.211538  10  +     10.0   
   208     Pb  126  184   6.10069  12+         1  0.211538  12  +     12.0   

        j_evenness  p_bit  quanta  energy_quantum        r2  energy_pred  
z  a                                                                      
3  6          True      0       0        1.114385  0.922325     0.000000  
   6          True      0       1        1.114385  0.922325     6.686312  
   6          True      0       2        1.114385  0.922325    22.287706  
5  10         True      0       0        0.325736  0.357721     0.000000  
   10         True      0       1        0.325736  0.357721     1.954414  
...            ...    ...     ...             ...       ...          ...  
82 208        True      0       2        0.048356 -0.598733     0.967112  
   208        True      0       3        0.048356 -0.598733     2.030935  
   208        True      0       4        0.048356 -0.598733     3.481603  
   208        True      0       5        0.048356 -0.598733     5.319115  
   208        True      0       6        0.048356 -0.598733     7.543472  

[1056 rows x 16 columns]

In [24]:
merged.loc[24,52]

symbol   n  idx    energy   jp  jp_order      beta   j  p  j_float  \
z  a                                                                       
24 52     Cr  28    0       0.0   0+         1  0.076923   0  +      0.0   
   52     Cr  28    1  1.434091   2+         1  0.076923   2  +      2.0   
   52     Cr  28    2   2.36963   4+         1  0.076923   4  +      4.0   
   52     Cr  28    6  3.113858   6+         1  0.076923   6  +      6.0   
   52     Cr  28   26   4.75031   8+         1  0.076923   8  +      8.0   
   52     Cr  28  125    7.2379  10+         1  0.076923  10  +     10.0   

       j_evenness  p_bit  quanta  energy_quantum        r2  energy_pred  
z  a                                                                     
24 52        True      0       0        0.068003  0.932256     0.000000  
   52        True      0       1        0.068003  0.932256     0.408018  
   52        True      0       2        0.068003  0.932256     1.360060  
   52        True      0       3        0.068003  0.932256     2.856126  
   52        True      0       4        0.068003  0.932256     4.896216  
   52        True      0       5        0.068003  0.932256     7.480330

In [25]:
df_to_plot = best_osc_merged
# df_to_plot = merged

fig = px.scatter(df_to_plot.groupby(by=df_to_plot.index).head(1).reset_index(), 
          # x='a', 
          x='n',
          y='z', 
          color='r2',
          title='R² Values for Best Oscillator Fits',
          # labels={'z': 'Number of protons (Z)', 'a': 'Weight (A)', 'r2': 'R² Score'},
          labels={'z': 'Number of protons (Z)', 'n': 'Number of neutrons (N)', 'r2': 'R² Score'},
          color_continuous_scale='RdBu',
          height=600)


magic_numbers_n = [2, 8, 20, 50, 58, 82, 126]
magic_numbers_z = [2, 8, 20, 50, 58, 82]

for i, m in enumerate(magic_numbers_n):
    fig.add_vline(x=m, line_dash="dash", line_color="gray", name="Magic numbers" if i==0 else None)

for i, m in enumerate(magic_numbers_z):
    fig.add_hline(y=m, line_dash="dash", line_color="gray")

fig.show()

In [27]:
df_to_plot = merged

px.scatter(df_to_plot.groupby(by=df_to_plot.index).head(1).reset_index(), 
            x='a', 
            y='r2', 
            color='beta',
            title='R² Values for All Oscillator Fits',
            labels={'beta': 'Asymmetry parameter', 'a': 'Weight (A)', 'r2': 'R² Score'},
            color_continuous_scale='viridis')

In [28]:
df_to_plot = merged

fig = px.scatter(df_to_plot.groupby(by=df_to_plot.index).head(1).reset_index(), 
          # x='a', 
          x='n',
          y='z', 
          color='r2',
          title='R² Values for All Oscillator Fits',
          # labels={'z': 'Number of protons (Z)', 'a': 'Weight (A)', 'r2': 'R² Score'},
          labels={'z': 'Number of protons (Z)', 'n': 'Number of neutrons (N)', 'r2': 'R² Score'},
          color_continuous_scale=[(0, 'red'), (0.8, 'white'), (1, 'blue')],
          height=600)


magic_numbers_n = [2, 8, 20, 50, 58, 82, 126]
magic_numbers_z = [2, 8, 20, 50, 58, 82]

for i, m in enumerate(magic_numbers_n):
    fig.add_vline(x=m, line_dash="dash", line_color="gray", name="Magic numbers" if i==0 else None)

for i, m in enumerate(magic_numbers_z):
    fig.add_hline(y=m, line_dash="dash", line_color="gray")

fig.show()

In [29]:
merged.loc[24,52]

symbol   n  idx    energy   jp  jp_order      beta   j  p  j_float  \
z  a                                                                       
24 52     Cr  28    0       0.0   0+         1  0.076923   0  +      0.0   
   52     Cr  28    1  1.434091   2+         1  0.076923   2  +      2.0   
   52     Cr  28    2   2.36963   4+         1  0.076923   4  +      4.0   
   52     Cr  28    6  3.113858   6+         1  0.076923   6  +      6.0   
   52     Cr  28   26   4.75031   8+         1  0.076923   8  +      8.0   
   52     Cr  28  125    7.2379  10+         1  0.076923  10  +     10.0   

       j_evenness  p_bit  quanta  energy_quantum        r2  energy_pred  
z  a                                                                     
24 52        True      0       0        0.068003  0.932256     0.000000  
   52        True      0       1        0.068003  0.932256     0.408018  
   52        True      0       2        0.068003  0.932256     1.360060  
   52        True      0       3        0.068003  0.932256     2.856126  
   52        True      0       4        0.068003  0.932256     4.896216  
   52        True      0       5        0.068003  0.932256     7.480330

In [30]:
test_nucleus = merged.loc[24,52]
a = test_nucleus.index[0][1]
symbol = test_nucleus.iloc[0]['symbol']

px.scatter(x=test_nucleus['quanta'], 
            y=test_nucleus['energy']-test_nucleus['energy_pred'],
            title=f'Residuals for {a}{symbol}',
            labels={'x': 'Quanta of oscillation', 'y': 'Residuals'})

In [31]:
residuals_df = best_osc_merged.copy()
residuals_df = residuals_df.reset_index()
residuals_df["name"] = residuals_df["a"].astype(str) + residuals_df["symbol"]
residuals_df["residuals"] = residuals_df["energy"] - residuals_df["energy_pred"]

best_residuals = get_best_r2(residuals_df, 0.99)

fig = px.scatter(best_residuals, x="quanta", y="residuals", facet_col="name", facet_col_wrap=8)
fig.add_hline(y=0, line_width=1, line_dash="dash", line_color="black")
fig.show()

In [47]:
test_nuclei = residuals_df[(residuals_df['a'] == 168) & (residuals_df['symbol'] == "Yb")]
fig = px.scatter(test_nuclei, x="quanta", y="energy")
fig.show()

In [48]:
gaps = np.diff(test_nuclei.iloc[:3]['energy'])
print(gaps[1] / gaps[0])

2.266282913484555


In [236]:
magic_numbers = [2, 8, 20, 50, 58, 82, 126]

magic_residuals_df = merged.copy().reset_index()
magic_residuals_df = magic_residuals_df[magic_residuals_df['z'].isin(magic_numbers) | magic_residuals_df['n'].isin(magic_numbers)]

magic_residuals_df["name"] = magic_residuals_df["a"].astype(str) + magic_residuals_df["symbol"]
magic_residuals_df["residuals"] = magic_residuals_df["energy"] - magic_residuals_df["energy_pred"]

best_magic_residuals = get_best_r2(magic_residuals_df, 0.90)

fig = px.scatter(best_magic_residuals, x="quanta", y="residuals", facet_row="z", facet_col="n", height=1000, color='r2')
fig.add_hline(y=0, line_width=1, line_dash="dash", line_color="black")
fig.show()

In [237]:
#What if any fit, not linear?
# Note some seem to have two different bands, (W shape)
# Note test multiple models, and only show best fit


# Just use even-even??

In [238]:
residuals_df[residuals_df['name'] == '104Pd']

,z,a,symbol,n,idx,energy,jp,jp_order,beta,j,...,j_evenness,p_bit,quanta,energy_quantum,zero_point_energy,r2,j_float_y,energy_pred,name,residuals
348,46,104,Pd,58,0,0.0,0+,1,0.115385,0,...,True,0,0,0.82376,-0.184872,0.997169,0,-0.184872,104Pd,0.184872
349,46,104,Pd,58,1,0.55581,2+,1,0.115385,2,...,True,0,1,0.82376,-0.184872,0.997169,1,0.638888,104Pd,-0.083078
350,46,104,Pd,58,2,1.32359,4+,1,0.115385,4,...,True,0,2,0.82376,-0.184872,0.997169,2,1.462649,104Pd,-0.139059
351,46,104,Pd,58,21,2.2495,6+,1,0.115385,6,...,True,0,3,0.82376,-0.184872,0.997169,3,2.286409,104Pd,-0.036909
352,46,104,Pd,58,88,3.2207,8+,1,0.115385,8,...,True,0,4,0.82376,-0.184872,0.997169,4,3.110170,104Pd,0.11053
353,46,104,Pd,58,134,4.0231,10+,1,0.115385,10,...,True,0,5,0.82376,-0.184872,0.997169,5,3.933930,104Pd,0.08917
354,46,104,Pd,58,138,4.635,12+,1,0.115385,12,...,True,0,6,0.82376,-0.184872,0.997169,6,4.757691,104Pd,-0.122691
355,46,104,Pd,58,141,5.4321,14+,1,0.115385,14,...,True,0,7,0.82376,-0.184872,0.997169,7,5.581451,104Pd,-0.149351
356,46,104,Pd,58,144,6.3583,16+,1,0.115385,16,...,True,0,8,0.82376,-0.184872,0.997169,8,6.405212,104Pd,-0.046912
357,46,104,Pd,58,145,7.4224,18+,1,0.115385,18,...,True,0,9,0.82376,-0.184872,0.997169,9,7.228972,104Pd,0.193428


In [239]:
df.loc[46,104]

symbol   n  idx   energy   jp  jp_order      beta   j  p  j_float  \
z  a                                                                       
46 104     Pd  58    0      0.0   0+         1  0.115385   0  +      0.0   
   104     Pd  58    1  0.55581   2+         1  0.115385   2  +      2.0   
   104     Pd  58    2  1.32359   4+         1  0.115385   4  +      4.0   
   104     Pd  58    3  1.33359   0+         2  0.115385   0  +      0.0   
   104     Pd  58    4  1.34168   2+         2  0.115385   2  +      2.0   
...       ...  ..  ...      ...  ...       ...       ...  .. ..      ...   
   104     Pd  58  141   5.4321  14+         1  0.115385  14  +     14.0   
   104     Pd  58  142   5.6812  14-         1  0.115385  14  -     14.0   
   104     Pd  58  143   6.0218  15-         1  0.115385  15  -     15.0   
   104     Pd  58  144   6.3583  16+         1  0.115385  16  +     16.0   
   104     Pd  58  145   7.4224  18+         1  0.115385  18  +     18.0   

        j_evenness  p_bit  
z  a                       
46 104        True      0  
   104        True      0  
   104        True      0  
   104        True      0  
   104        True      0  
...            ...    ...  
   104        True      0  
   104        True      1  
   104       False      1  
   104        True      0  
   104        True      0  

[75 rows x 12 columns]

In [240]:
#TODO compare with paper, check their r2 for "perfect" and for "deviation"
#TODO look at paper?

In [241]:
# Using r2 not mse because scale independent, some energy levels may be inherently larger?
#https://stats.stackexchange.com/a/250735

In [242]:
#TODO figure out way to combine r2 and number of points used

In [243]:
#Quadrupole axial: Even positive only?

#TODO use bands